In [23]:
import torch
import torch.nn as nn
import numpy as np
from tqdm import tqdm
from torchvision.utils import make_grid
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
from PIL import Image
import os
import matplotlib.pyplot as plt
import torch.nn.functional as F
from tqdm import tqdm
from torchvision.utils import save_image

In [24]:
class UNetForUpsampling(nn.Module):
    def __init__(self):
        super(UNetForUpsampling, self).__init__()
        self.encoder1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True)
        )
        self.encoder2 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1),
            nn.ReLU(inplace=True)
        )

        # Only two encoder stages due to smaller input size
        self.middle = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=3, stride=2, padding=1),
            nn.ReLU(inplace=True)
        )

        self.decoder2 = nn.Sequential(
            nn.ConvTranspose2d(256, 128, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.ReLU(inplace=True)
        )

        # Adjusting the decoder to match the target upsampling size
        self.decoder1 = nn.Sequential(
            nn.ConvTranspose2d(128*2, 64, kernel_size=2, stride=2),  # kernel size and stride adjusted for upscaling
            nn.ReLU(inplace=True)
        )

        # Final layer to upscale from current size to desired 48x48 output
        self.upsample = nn.Upsample(size=(256, 256), mode='bicubic', align_corners=False)
        self.final = nn.Conv2d(128, 3, kernel_size=3, stride=1, padding=1)

    def forward(self, x):
        x1 = self.encoder1(x)
        x2 = self.encoder2(x1)

        xm = self.middle(x2)

        d2 = self.decoder2(xm)
        d2 = torch.cat((x2, d2), dim=1)  # Skip connection
        d1 = self.decoder1(d2)
        d1 = torch.cat((x1, d1), dim=1)  # Skip connection

        upsampled = self.upsample(d1)  # Upsample to target size
        out = self.final(upsampled)
        return out

In [25]:
import os
from PIL import Image
from torch.utils.data import Dataset, DataLoader, random_split
import torchvision.transforms as transforms
import torch

# -------- Dataset Definition --------
class ImagePairDataset(Dataset):
    def __init__(self, lr_folder, hr_folder, lr_transform=None, hr_transform=None):
        self.lr_folder = lr_folder
        self.hr_folder = hr_folder
        self.lr_transform = lr_transform
        self.hr_transform = hr_transform

        # Only use image names that are in both folders
        self.image_names = list(set(os.listdir(lr_folder)) & set(os.listdir(hr_folder)))
        self.image_names = [img for img in self.image_names if img.endswith('.png')]

    def __len__(self):
        return len(self.image_names)

    def __getitem__(self, idx):
        image_name = self.image_names[idx]
        lr_image_path = os.path.join(self.lr_folder, image_name)
        hr_image_path = os.path.join(self.hr_folder, image_name)

        lr_image = Image.open(lr_image_path).convert('RGB')
        hr_image = Image.open(hr_image_path).convert('RGB')

        if self.lr_transform:
            lr_image = self.lr_transform(lr_image)
        if self.hr_transform:
            hr_image = self.hr_transform(hr_image)

        return lr_image, hr_image

In [26]:

# -------- Folder Paths --------
folder_path_train_hr = '/home/vittorio/Documenti/Upsampling_CFD/datasets/SplitDatasetOutdoorFlow/train/high_res'
folder_path_train_lr = '/home/vittorio/Documenti/Upsampling_CFD/datasets/SplitDatasetOutdoorFlow/train/low_res'
folder_path_val_hr =  '/home/vittorio/Documenti/Upsampling_CFD/datasets/SplitDatasetOutdoorFlow/test/high_res'
folder_path_val_lr ='/home/vittorio/Documenti/Upsampling_CFD/datasets/SplitDatasetOutdoorFlow/test/low_res'

# -------- Transforms --------
lr_transform = transforms.Compose([
    #transforms.Resize((16, 16)),  # Adjust as needed
    transforms.ToTensor(),
])

hr_transform = transforms.Compose([
    transforms.ToTensor(),  # Assumes HR is already at correct resolution
])


In [27]:


# -------- Create Datasets --------
full_train_dataset = ImagePairDataset(folder_path_train_lr, folder_path_train_hr,
                                      lr_transform=lr_transform, hr_transform=hr_transform)

test_dataset = ImagePairDataset(folder_path_val_lr, folder_path_val_hr,
                                 lr_transform=lr_transform, hr_transform=hr_transform)

# -------- DataLoaders --------
train_loader = DataLoader(full_train_dataset, batch_size=8, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=8, shuffle=False)

# -------- Print Stats --------
print(f"Train Dataset Size: {len(full_train_dataset)}")
print(f"Test Dataset Size: {len(test_dataset)}")


Train Dataset Size: 772
Test Dataset Size: 192


In [28]:
num_epochs = 80
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
# model = UpsampleDiffusionModel().to(device)
model = UNetForUpsampling().to(device)
# model = UNetForUpsamplingGELU().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
criterion = torch.nn.MSELoss()

train_losses = []
val_losses = []

for epoch in tqdm(range(num_epochs)):
    model.train()  # Set model to training mode

    # Training phase
    total_train_loss = 0
    for lr_imgs, hr_imgs in train_loader:
        lr_imgs, hr_imgs = lr_imgs.to(device), hr_imgs.to(device)
        optimizer.zero_grad()
        outputs = model(lr_imgs)
        loss = criterion(outputs, hr_imgs)
        loss.backward()
        optimizer.step()
        total_train_loss += loss.item()
    avg_train_loss = total_train_loss / len(train_loader)

    # Validation phase
    model.eval()  # Set model to evaluation mode
    total_val_loss = 0
    with torch.no_grad():  # No gradients needed for validation
        for lr_imgs, hr_imgs in test_loader:
            lr_imgs, hr_imgs = lr_imgs.to(device), hr_imgs.to(device)
            outputs = model(lr_imgs)
            loss = criterion(outputs, hr_imgs)
            total_val_loss += loss.item()
    avg_val_loss = total_val_loss / len(test_loader)

    # Save the average losses
    train_losses.append(avg_train_loss)
    val_losses.append(avg_val_loss)

    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}')

cuda


  1%|▏         | 1/80 [00:08<11:39,  8.86s/it]

Epoch [1/80], Train Loss: 0.1209, Val Loss: 0.0150


  2%|▎         | 2/80 [00:16<10:56,  8.42s/it]

Epoch [2/80], Train Loss: 0.0077, Val Loss: 0.0043


...

Epoch [78/80], Train Loss: 0.0003, Val Loss: 0.0003


 99%|█████████▉| 79/80 [12:27<00:09,  9.65s/it]

Epoch [79/80], Train Loss: 0.0003, Val Loss: 0.0003


100%|██████████| 80/80 [12:37<00:00,  9.46s/it]

Epoch [80/80], Train Loss: 0.0003, Val Loss: 0.0003


In [29]:
import torch
import torch.nn.functional as F

def psnr(pred, target, max_val=1.0):
    mse = F.mse_loss(pred, target)
    return 20 * torch.log10(max_val / torch.sqrt(mse))

def calculate_average_psnr(data_loader, model, device):
    total_psnr_model = 0.0
    total_batches = 0

    model.eval()
    with torch.no_grad():
        for lr_imgs, hr_imgs in data_loader:
            lr_imgs = lr_imgs.to(device)
            hr_imgs = hr_imgs.to(device)

            # Forward pass
            sr_imgs = model(lr_imgs)

            # Optional visualization (only on the first sample in batch)
            # visualize_sr_vs_hr(sr_imgs[0].cpu(), hr_imgs[0].cpu())

            # Compute PSNR directly
            psnr_model = psnr(sr_imgs, hr_imgs).item()
            total_psnr_model += psnr_model
            total_batches += 1

    avg_psnr_model = total_psnr_model / total_batches
    return avg_psnr_model

val_psnr_model = calculate_average_psnr(test_loader, model, device)
print(f"Validation PSNR - Model Output: {val_psnr_model:.2f}")


Validation PSNR - Model Output: 34.91


In [30]:
import torch
import torch.nn.functional as F

def ssim(pred, target, C1=0.01**2, C2=0.03**2):
    """Structural Similarity Index Measure (SSIM) for image batches."""
    # Assumes pred and target are shape (B, C, H, W)
    B, C, H, W = pred.size()

    # Create a Gaussian filter manually (uniform box blur here for simplicity)
    kernel = torch.ones((C, 1, 11, 11), device=pred.device) / 121

    # Compute means
    mu_pred = F.conv2d(pred, kernel, padding=5, groups=C)
    mu_target = F.conv2d(target, kernel, padding=5, groups=C)

    # Compute variances and covariance
    sigma_pred = F.conv2d(pred * pred, kernel, padding=5, groups=C) - mu_pred ** 2
    sigma_target = F.conv2d(target * target, kernel, padding=5, groups=C) - mu_target ** 2
    sigma_cross = F.conv2d(pred * target, kernel, padding=5, groups=C) - mu_pred * mu_target

    # SSIM formula
    numerator = (2 * mu_pred * mu_target + C1) * (2 * sigma_cross + C2)
    denominator = (mu_pred ** 2 + mu_target ** 2 + C1) * (sigma_pred + sigma_target + C2)
    ssim_map = numerator / denominator

    return ssim_map.mean()

def calculate_average_ssim(data_loader, model, device):
    total_ssim_model = 0.0
    total_ssim_bicubic = 0.0
    total_batches = 0

    model.eval()
    with torch.no_grad():
        for lr_imgs, hr_imgs in data_loader:
            lr_imgs = lr_imgs.to(device)
            hr_imgs = hr_imgs.to(device)

            # Get model output
            sr_imgs = model(lr_imgs)
            sr_imgs = torch.clamp(sr_imgs, 0.0, 1.0)

            # Bicubic baseline
            bicubic = F.interpolate(lr_imgs, scale_factor=4, mode='bicubic', align_corners=False)
            bicubic = torch.clamp(bicubic, 0.0, 1.0)

            # Compute SSIM
            ssim_model = ssim(sr_imgs, hr_imgs).item()
            ssim_bicubic = ssim(bicubic, hr_imgs).item()

            total_ssim_model += ssim_model
            total_ssim_bicubic += ssim_bicubic
            total_batches += 1

    avg_ssim_model = total_ssim_model / total_batches
    avg_ssim_bicubic = total_ssim_bicubic / total_batches
    return avg_ssim_model, avg_ssim_bicubic


val_ssim_model, val_ssim_bicubic = calculate_average_ssim(test_loader, model, device)
print(f"Validation SSIM - Model Output: {val_ssim_model:.4f}, Bicubic: {val_ssim_bicubic:.4f}")


RuntimeError: The size of tensor a (512) must match the size of tensor b (256) at non-singleton dimension 3

In [ ]:
import torch
import torch.nn.functional as F

def compute_rmse(pred, target):
    """
    Compute Root Mean Squared Error (RMSE) between prediction and ground truth.
    Assumes both are tensors in [0, 1] and of shape (B, C, H, W).
    """
    mse = F.mse_loss(pred, target)
    return torch.sqrt(mse).item()

def calculate_average_rmse(data_loader, model, device):
    """
    Compute average RMSE for model predictions and bicubic interpolation over a dataset.
    """
    total_rmse_model = 0.0
    total_rmse_bicubic = 0.0
    total_batches = 0

    model.eval()
    with torch.no_grad():
        for lr_imgs, hr_imgs in data_loader:
            lr_imgs = lr_imgs.to(device)
            hr_imgs = hr_imgs.to(device)

            # Model prediction
            sr_imgs = model(lr_imgs)
            sr_imgs = torch.clamp(sr_imgs, 0.0, 1.0)

            # Bicubic interpolation baseline
            bicubic = F.interpolate(lr_imgs, scale_factor=4, mode='bicubic', align_corners=False)
            bicubic = torch.clamp(bicubic, 0.0, 1.0)

            # Compute RMSEs
            rmse_model = compute_rmse(sr_imgs, hr_imgs)
            rmse_bicubic = compute_rmse(bicubic, hr_imgs)

            total_rmse_model += rmse_model
            total_rmse_bicubic += rmse_bicubic
            total_batches += 1

    avg_rmse_model = total_rmse_model / total_batches
    avg_rmse_bicubic = total_rmse_bicubic / total_batches

    return avg_rmse_model, avg_rmse_bicubic

# Assuming your DataLoaders and model are ready
val_rmse_model, val_rmse_bicubic = calculate_average_rmse(test_loader, model, device)

print(f"Validation RMSE - Model Output: {val_rmse_model:.4f}, Bicubic: {val_rmse_bicubic:.4f}")


Validation RMSE - Model Output: 0.0402, Bicubic: 0.0607


: 

In [ ]:
def show_images_from_dataset(dataset_type='test', batch_num=0, num_images=4, first_image=0):
    # Select the appropriate DataLoader
    if dataset_type == 'train':
        selected_loader = train_loader
    elif dataset_type == 'val':
        selected_loader = val_loader
    else:
        raise ValueError("Invalid dataset type. Choose 'train', 'val', or 'test'.")

    model.eval()  # Set the model to evaluation mode

    current_batch = 0
    with torch.no_grad():  # Inference without gradient calculation
        for lr_imgs, hr_imgs in selected_loader:
            if current_batch == batch_num:
                lr_imgs, hr_imgs = lr_imgs.to(device), hr_imgs.to(device)
                outputs = model(lr_imgs)
                bicubic = torch.nn.functional.interpolate(lr_imgs, scale_factor=2, mode='bicubic', align_corners=False)
                break  # Exit loop after processing the specified batch
            current_batch += 1

    # Visualization
    plt.figure(figsize=(10, 10))
    for i in range(min(num_images, lr_imgs.size(0))):  # Ensure not to exceed batch size
        # Plotting code remains the same as before
        # Low-resolution images
        plt.subplot(4, num_images, i+1)
        plt.imshow(lr_imgs[i+first_image].permute(1, 2, 0).cpu().detach().numpy())
        plt.title('Low-Res')
        plt.axis('off')

        # High-resolution (Ground Truth)
        plt.subplot(4, num_images, num_images+i+1)
        plt.imshow(hr_imgs[i+first_image].permute(1, 2, 0).cpu().detach().numpy())
        plt.title('High-Res')
        plt.axis('off')

        # Model output
        plt.subplot(4, num_images, 2*num_images+i+1)
        plt.imshow(outputs[i+first_image].permute(1, 2, 0).cpu().detach().numpy())
        plt.title('Output')
        plt.axis('off')

        # Bicubic upsampled
        plt.subplot(4, num_images, 3*num_images+i+1)
        plt.imshow(bicubic[i+first_image].permute(1, 2, 0).cpu().detach().numpy())
        plt.title('Bicubic')
        plt.axis('off')
    plt.show()

# Example usage:
show_images_from_dataset(dataset_type='val', batch_num=3, num_images=4, first_image=20)
print(f"Number of batches in Training DataLoader: {len(train_loader)}")
print(f"Number of batches in Validation DataLoader: {len(val_loader)}")
print(lr_imgs.size())
print(hr_imgs.size())
print(outputs.size())
